In [22]:
import pandas as pd
df=pd.read_csv('train.csv')
df.head()

,label,news
0,False,Says the Annies List political group supports ...
1,True,When did the decline of coal start? It started...
2,True,"Hillary Clinton agrees with John McCain ""by vo..."
3,False,Health care reform legislation is likely to ma...
4,True,The economic turnaround started at the end of ...


In [2]:
###Drop Nan Values
df=df.dropna()
## Get the Independent Features

X=df.drop('label',axis=1)

In [16]:
y=df['label']
X.shape

(10240,)

In [5]:
y.shape

(10240,)

In [8]:
import tensorflow as tf
tf.__version__


'2.20.0-rc0'

In [62]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [15]:
voc_size=5000

In [36]:
messages=X.copy()
print(messages)

0        Says the Annies List political group supports ...
1        When did the decline of coal start? It started...
2        Hillary Clinton agrees with John McCain "by vo...
3        Health care reform legislation is likely to ma...
4        The economic turnaround started at the end of ...
                               ...                        
10235    There are a larger number of shark attacks in ...
10236    Democrats have now become the party of the [At...
10237    Says an alternative to Social Security that op...
10238    On lifting the U.S. Cuban embargo and allowing...
10239    The Department of Veterans Affairs has a manua...
Name: news, Length: 10240, dtype: object


In [38]:
print(messages[1])

When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.


In [42]:
messages = df[['news']]  # double brackets to keep it as DataFrame
messages.reset_index(inplace=True)

In [44]:
messages['news'][1]

'When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.'

In [46]:
import nltk
import re
from nltk.corpus import stopwords

In [47]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Sakshath
[nltk_data]     Shetty\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [50]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['news'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [59]:
from tensorflow.keras.preprocessing.text import one_hot
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

In [55]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4084 2498 2182]
 [   0    0    0 ... 2682 3935  873]
 [   0    0    0 ... 1589 2098 1571]
 ...
 [   0    0    0 ... 3238 3165   38]
 [   0    0    0 ... 3196 2358 2151]
 [   0    0    0 ... 2157   21 2591]]


In [57]:
embedded_docs[2]


array([   0,    0,    0,    0,    0,    0,    0,    0,  515, 1692, 1243,
       4061, 1713, 3881, 2386, 2356, 3935, 1589, 2098, 1571], dtype=int32)

In [64]:

embedding_vector_features=40
model=Sequential()
model.add(Embedding(input_dim=voc_size, output_dim=embedding_vector_features, input_shape=(sent_length,)))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.build(input_shape=(None, sent_length))
print(model.summary())

E:\work\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 100)                 │          56,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

None


In [65]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [66]:
import numpy as np

X_train = np.array(embedded_docs)  # processed training titles
y_train = np.array()         # corresponding labels (0/1)


NameError: name 'train_embedded_docs' is not defined

In [ ]:
# Load test CSV
test_df = pd.read_csv("test_lower.csv")  

# Extract text and labels
test_sentences = test_df["news"]
test_labels = test_df["label"]

# Tokenize + pad
test_onehot_repr = [one_hot(words, voc_size) for words in test_sentences]
test_embedded_docs = pad_sequences(test_onehot_repr, padding='pre', maxlen=sent_length)

# Convert to numpy
X_test = np.array(test_embedded_docs)
y_test = np.array(test_labels)